## Stage 03: Binomial Tree Model
- Maturity (T): 1-5 trading days
- Strikes (K): based on today's close price (adj_close), with offsets: {-$2, -$1, $0, +$1, +$2}
- Outputs: Call & Put prices for each (T, K) pair
- Models: Binomial Tree

In [10]:
# project path setup
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

### Load the latest price file and get ticker and close price

In [26]:
from pathlib import Path
import pandas as pd

# Find the latest raw daily price CSV file in data/raw
raw_dir = PROJECT_ROOT / "data" / "raw"
files = sorted(raw_dir.glob("*_daily_*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)

if not files:
    raise FileNotFoundError("No *_daily_*.csv found in data/raw")

latest_file = files[0]
print("Latest price file: ", latest_file)

# Read the latest file
price_df = pd.read_csv(latest_file)

# Ensure date is datetime and sort
price_df["date"] = pd.to_datetime(price_df["date"])
price_df = price_df.sort_values("date").reset_index(drop=True)

# Pick price column (prepfer adj_close)
price_col = "adj_close" if "adj_close" in price_df.columns else "close"

# Today inputs
ticker = latest_file.name.split("_")[0]
asof_date = price_df["date"].iloc[-1].date()
S0 = float(price_df[price_col].iloc[-1])

print("Ticker: ", ticker)
print("As-of date", asof_date)
print("S0 (latest adj_close/close): ", S0)

Latest price file:  /Users/allenpilipala/BSM_model/data/raw/AAPL_yfinance_daily_2024-01-08_2026-01-08.csv
Ticker:  AAPL
As-of date 2026-01-07
S0 (latest adj_close/close):  260.3299865722656


### Load Volatility Table and build a sigma lookup

In [29]:
import numpy as np

vol_path = PROJECT_ROOT / "data" / "processed" / "volatility_inputs.csv"
vol_df = pd.read_csv(vol_path)

# Ensure consistent types
vol_df["horizon_days"] = vol_df["horizon_days"].astype(int)

# Filter to current ticker
vol_df = vol_df[vol_df["ticker"] == ticker].copy()

if vol_df.empty:
    raise ValueError(f"No volatility rows found for ticker == {ticker} in {vol_path}")

print(vol_df.head(10))

  ticker         model  horizon_days     sigma  annualized date_generated
0   AAPL         garch             1  0.014742       False     2026-01-08
1   AAPL         garch             2  0.021167       False     2026-01-08
2   AAPL         garch             3  0.026257       False     2026-01-08
3   AAPL         garch             4  0.030648       False     2026-01-08
4   AAPL         garch             5  0.034584       False     2026-01-08
5   AAPL  hist_rolling             1  0.007680       False     2026-01-08
6   AAPL  hist_rolling             2  0.010861       False     2026-01-08
7   AAPL  hist_rolling             3  0.013302       False     2026-01-08
8   AAPL  hist_rolling             4  0.015360       False     2026-01-08
9   AAPL  hist_rolling             5  0.017173       False     2026-01-08


### Convert horizon sigma to annualized sigma

In [33]:
def to_annualized_sigma(sigma_h, d, trading_days=252):
    # sigma_h = volatility over d days
    # Convert to annualized volatility
    return sigma_h / np.sqrt(d / trading_days)

# Add sigma_ann to table
vol_df["sigma_ann"] = vol_df.apply(
    lambda row: to_annualized_sigma(row["sigma"], int(row["horizon_days"])),
    axis=1
)

# Build lookups: sigma_ann_lookup[model][d] = sigma_ann
sigma_ann_lookup = {}
for model in vol_df["model"].unique():
    sub = vol_df[vol_df["model"] == model]
    sigma_ann_lookup[model] = dict(zip(sub["horizon_days"], sub["sigma_ann"]))

sigma_ann_lookup


{'garch': {1: 0.23402978316872058,
  2: 0.23759866846878286,
  3: 0.24064666197616807,
  4: 0.24326321581035038,
  5: 0.24552022087108402},
 'hist_rolling': {1: 0.12191417121572552,
  2: 0.12191417121572609,
  3: 0.12191417121572565,
  4: 0.12191417121572633,
  5: 0.1219141712157264}}

### Find Risk-free rate
We proxy the short-term risk-free rate using the 3-month Treasury Bill yield (^IRX) obtained from Yahoo Finance, which closely approximates the risk-free rate for very short-dated options.

In [43]:
import yfinance as yf

import yfinance as yf

irx = yf.download("^IRX", period="1mo", progress=False)
irx = irx.reset_index()

irx["r_annual"] = irx["Close"] / 100
irx = irx[["Date", "r_annual"]]

irx.tail(5)


Price,Date,r_annual
Ticker,,
16,2025-12-31,0.03547
17,2026-01-02,0.03533
18,2026-01-05,0.03515
19,2026-01-06,0.03520
20,2026-01-07,0.03515
